In [1]:
import pandas as pd
from sqlalchemy import create_engine


### Store CSV into DataFrame

In [2]:
from secrets import username, password


In [3]:
csv_file = "../kroger/kroger_milk_prices.csv"
kroger_data_df = pd.read_csv(csv_file)
kroger_data_df.head()


,Unnamed: 0,productId,categories,brand,description,size,price.regular,price.promo,Date,Store_Id,image
0,0,1111040101,['Dairy'],Kroger,Kroger® Vitamin D Whole Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
1,1,1111041600,['Dairy'],Kroger,Kroger® 2% Reduced Fat Milk,1/2 gal,1.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
2,2,1111041660,['Dairy'],Kroger,Kroger® 1% Lowfat Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
3,3,1111040601,['Dairy'],Kroger,Kroger® Vitamin D Whole Milk,1/2 gal,1.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
4,4,1111042315,['Dairy'],Kroger,Kroger® Fat Free Skim Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...


In [4]:
milk_df = kroger_data_df
milk_df = milk_df.drop(columns=['Unnamed: 0', 'description' , 'price.regular', 'price.promo', 'Date'])
features = 'none'
milk_df['features'] = features
milk_df = milk_df.rename(columns={"productId": "product_id", "brand": "Brand" , "categories" : "category" , 'Store_Id' : 'Store_Number'})
milk_df['Type_ID'] = 1
milk_df.head()

,product_id,category,Brand,size,Store_Number,image,features,Type_ID
0,1111040101,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1
1,1111041600,['Dairy'],Kroger,1/2 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1
2,1111041660,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1
3,1111040601,['Dairy'],Kroger,1/2 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1
4,1111042315,['Dairy'],Kroger,1 gal,3400312,https://www.kroger.com/product/images/xlarge/f...,none,1


In [5]:
store_df = pd.DataFrame()
store_id = 3400312 , 1111111
store_df["Store_Number"] = store_id
zipcode = 77007 , 77007
store_df['Store_Zipcode'] = zipcode
name = "Kroger" , "HEB"
store_df['Store_Name'] = name
store_df.head()

,Store_Number,Store_Zipcode,Store_Name
0,3400312,77007,Kroger
1,1111111,77007,HEB


In [6]:
milk_type_df = pd.DataFrame()
milk_type = ['1%' , "2%", 'whole', 'skim', 'almond', 'other']
milk_type_df['Type'] = milk_type
milk_type_df.head()

,Type
0,1%
1,2%
2,whole
3,skim
4,almond


### Connect to local database

In [7]:
rds_connection_string = f"{username}:{password}@localhost:5432/milk_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [8]:
engine.table_names()

['Store', 'Milk', 'Milk_Type', 'Price_History']

Use drop duplicates to only add updated data to a dataframe that is then loaded into the database

In [9]:
temp_store = pd.read_sql_query('select * from "Store"', con=engine)
temp_store = temp_store.drop(columns = 'id')
#temp_store

In [10]:
db_store = pd.concat([temp_store, store_df]).drop_duplicates(keep=False)
db_store = db_store.reset_index(drop=True)   
#db_store.head()

### Use pandas to load csv converted DataFrame into database

In [11]:
db_store.to_sql(name='Store', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [12]:
pd.read_sql_query('select * from "Store"', con=engine).head()

,id,Store_Number,Store_Name,Store_Zipcode
0,1,3400312,Kroger,77007
1,2,1111111,HEB,77007


Same process as above for store table is followed for milk type.

In [13]:
temp_type = pd.read_sql_query('select * from "Milk_Type"', con=engine)
temp_type = temp_type.drop(columns = 'id')
db_type = pd.concat([temp_type, milk_type_df]).drop_duplicates(keep=False)
db_type = db_type.reset_index(drop=True)   
#db_type

In [14]:
db_type.to_sql(name='Milk_Type', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_location table

In [15]:
pd.read_sql_query('select * from "Milk_Type"', con=engine)

,id,Type
0,1,1%
1,2,2%
2,3,whole
3,4,skim
4,5,almond
5,6,other


Retrieves and merges Store data with milk data to set the Store ID then next cell deletes data not required in Milk Table

In [16]:
stores2_df = pd.read_sql_query('select * from "Store"', con=engine).head()
milk_df = milk_df.merge(stores2_df, on = 'Store_Number')
#milk_df.head()

In [17]:
milk_df = milk_df.drop(columns = ['Store_Name' , 'Store_Zipcode', 'Store_Number'])
#milk_df.head()

In [18]:
milk_df = milk_df.rename(columns ={'id' : 'Store_ID'})
#milk_df.head()

Same process as above is followed to insert Milk data from dataframe into Milk table of database

In [19]:
temp_milk = pd.read_sql_query('select * from "Milk"', con=engine)
temp_milk = temp_milk.drop(columns = 'id')
#temp_milk

In [20]:
db_milk = pd.concat([temp_milk, milk_df]).drop_duplicates(keep=False)
db_milk = db_milk.reset_index(drop=True)   
db_milk.head()

,product_id,Brand,features,size,category,image,Store_ID,Type_ID
0,1111040101,Kroger,none,1 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
1,1111041600,Kroger,none,1/2 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
2,1111041660,Kroger,none,1 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
3,1111040601,Kroger,none,1/2 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
4,1111042315,Kroger,none,1 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1


In [21]:
db_milk.to_sql(name='Milk', con=engine, if_exists='append', index=False)

In [22]:
pd.read_sql_query('select * from "Milk"', con=engine)

,id,product_id,Brand,features,size,category,image,Store_ID,Type_ID
0,1,1.111040e+09,Kroger,none,1 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
1,2,1.111042e+09,Kroger,none,1/2 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
2,3,1.111042e+09,Kroger,none,1 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
3,4,1.111041e+09,Kroger,none,1/2 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
4,5,1.111042e+09,Kroger,none,1 gal,['Dairy'],https://www.kroger.com/product/images/xlarge/f...,1,1
...,...,...,...,...,...,...,...,...,...
995,995,5.025521e+09,Ritter Sport,none,3.5 oz,['Snacks'],https://www.kroger.com/product/images/xlarge/f...,1,1
996,996,4.126038e+09,Simple Truth Organic,none,23.2 oz,['Baby'],https://www.kroger.com/product/images/xlarge/f...,1,1
997,998,8.206452e+10,Carol's Daughter,none,10 fl oz,"['Beauty', 'Personal Care']",https://www.kroger.com/product/images/xlarge/f...,1,1
998,999,7.928500e+10,Burt's Bees,none,12 fl oz,['Personal Care'],https://www.kroger.com/product/images/xlarge/f...,1,1


Milk data is pulled from database Milk table and merged with price history. Uneeded price history columns are dropped and renamed to coordinate with Price History Table. This sets the product ID in price history.

In [23]:
price_history_df = kroger_data_df[['Date', 'price.regular', 'price.promo', 'productId']].copy()
price_history_df =price_history_df.rename(columns={'Date':'date', 'price.regular':'price', 'price.promo':'saleprice', 'productId': 'product_id'})
price_history_df

,date,price,saleprice,product_id
0,02/08/2021,2.99,0.00,1111040101
1,02/08/2021,1.99,0.00,1111041600
2,02/08/2021,2.99,0.00,1111041660
3,02/08/2021,1.99,0.00,1111040601
4,02/08/2021,2.99,0.00,1111042315
...,...,...,...,...
995,02/08/2021,19.99,0.00,4126037569
996,02/08/2021,NaN,NaN,2279691008
997,02/08/2021,12.49,9.37,82064522612
998,02/08/2021,9.99,0.00,79285000664


In [24]:
milk2_df = pd.read_sql_query('select * from "Milk"', con=engine)
milk2_df = milk2_df.drop(columns = ['Brand' , 'size', 'image', 'Store_ID', 'Type_ID', 'category', 'features'])

merged_price = price_history_df.merge(milk2_df, on = 'product_id')
merged_price

,date,price,saleprice,product_id,id
0,02/08/2021,2.99,0.00,1111040101,1
1,02/08/2021,1.99,0.00,1111041600,2
2,02/08/2021,2.99,0.00,1111041660,3
3,02/08/2021,1.99,0.00,1111040601,4
4,02/08/2021,2.99,0.00,1111042315,5
...,...,...,...,...,...
995,02/08/2021,19.99,0.00,4126037569,996
996,02/08/2021,NaN,NaN,2279691008,997
997,02/08/2021,12.49,9.37,82064522612,998
998,02/08/2021,9.99,0.00,79285000664,999


In [25]:
merged_price = merged_price.drop(columns = ['product_id'])
merged_price = merged_price.rename(columns = {'id' : 'product_id'})
merged_price

,date,price,saleprice,product_id
0,02/08/2021,2.99,0.00,1
1,02/08/2021,1.99,0.00,2
2,02/08/2021,2.99,0.00,3
3,02/08/2021,1.99,0.00,4
4,02/08/2021,2.99,0.00,5
...,...,...,...,...
995,02/08/2021,19.99,0.00,996
996,02/08/2021,NaN,NaN,997
997,02/08/2021,12.49,9.37,998
998,02/08/2021,9.99,0.00,999


Price History Dataframe is then added to Price History Table. Duplicates are not checked for in this instance because duplicates will trigger an error and stop the update process. 

In [26]:
merged_price.to_sql(name='Price_History', con=engine, if_exists='append', index=False)

In [27]:
pd.read_sql_query('select * from "Price_History"', con=engine)

,product_id,date,saleprice,price
0,1,2021-02-08,0.00,2.99
1,2,2021-02-08,0.00,1.99
2,3,2021-02-08,0.00,2.99
3,4,2021-02-08,0.00,1.99
4,5,2021-02-08,0.00,2.99
...,...,...,...,...
995,996,2021-02-08,0.00,19.99
996,997,2021-02-08,NaN,NaN
997,998,2021-02-08,9.37,12.49
998,999,2021-02-08,0.00,9.99
